In [1]:
from IPython.display import display, Math, Latex
from IPython.core.display import HTML 

# Введение в теорию или как связаны подгузники и пиво?

   Обучение на ассоциативных правилах (далее Assocoations rules learning - ARL) представляет из себя с одной стороны, простой, с другой - довольно часто применимый в реальной жизни метод поиска взаимосвязей (ассоциаций) в датасетах или, если точнее, айтемсетах (itemsests). Именно ARL лежит в основе многих рекомендательных систем, работающих в интернет-магазинах. 
   Впервые подробно об этом заговорил Piatesky-Shapiro G [1] в работе “Discovery, Analysis, and Presentation of Strong Rules.” (1991) Более подробо тему развивали Agrawal R, Imielinski T, Swami A в работах “Mining Association Rules between Sets of Items in Large Databases” (1993) [2] и “Fast Algorithms for Mining Association Rules.” (1994) [3].

    
   


   
  В общем виде ARL можно опсиать как "Who bought x also bought y" ("Кто купил x, также купил y"). В основе лежит анализ транзакций (transactions), внутри каждой из которых лежит свой уникальный itemset из набора items. При помощи ARL алогритмов нахоядтся те самые "правила" совпаденяи items внутри одной транзакции, котоыре потом сортируются по их силе. 

   Да, вот все так просто и логично.

   За этой простотой, однако, могут скрываться поразительные вещи, о которых common sense даже не подозревал:) 

   Классический случай такого когнитивного диссонанса описан в статье D.J. Power "Ask Dan!", опубликованной в DSSResources.com [4]: 

   В 1992 году, группа по консалтингу в области ритейла компании Teradata под руководством Томаса Блишока провела исследование 1.2 миллиона транзакций в 25 магазинах для ритейлера Osco Drug (нет, там продавали не наркотики и даже не лекарства, точнее, не только лекартсва. Drug Store в стране веротяного противника - формат разнокалиберных магазинов у дома). 
   После анализа всех этих транзакций самым сильным правилом получилось "Между 17:00 и 19:00 чаще всего пиво и подгузники покупают вместе". 
   К сожалению такое правило показалось руководству Osco Drug настолкьо контринтуитивным, что ставить подгузники на полках рядом с пивом они не стали:) Хотя объяснение паре пиво-подгузники вполне себе нашлось: когда оба члена молодой семьи возвращались с рабоыт домой (как раз часам к 5 вечера), жены обычно отправляли мужей за подгузниками в ближайшей магазин. И мужья, не долго думая, совмещали приятное с полезным - покупали подгузники по заданию жены и пиво для собственного вечернего времяпрепровождения

# Описание Association rule

Итак мы выяснили, что пиво и подгузники - хороший джентельменский набор, а что дальше? 

Пусть у нас имеется некий датасет (или коллекция) D, такой, что D = {d_0 ... d_j}, где d - уникальная транзакция-itemset (например, кассовый чек).
Внутри каждой d представлен набор items (i - item), причем в идеально случае он представлен в бинарном виде: d1 = [{Пиво: 1}, {Вода: 0}, {Кола: 1}, {...}], d2 = [{Пиво : 0}, {Вода: 1}, {Кола : 1}, {...}]. Принято каждый itemset описывать через количество ненулевых значений (k-itemset), например, [{Пиво: 1}, {Вода: 0}, {Кола: 1}] являестя 2-itemset.

Если в изначальном виде не представлен, можно при желании руками его преобразовать (OHE и pd.get_dummies вам в помощь).

Таким образом, датасет представляет собой разреженную матрицу со значениями {1,0}. Это будет бинарный датасет. Существуют и другие виды записи - вертикальный датасет и транзакционный датасет

ОК, данные преобразовали, как найти правила?

Существует целый ряд ключевых (если хотите - базовых) понятий в ARL, которые нам помогут эти правила вывести.

##### Support (поддержка)

Первое понятие  в ARL - support:


$supp(X) = \frac{\{t\in T;\ X \in t\}}{|T|}$

где X - itemset, содержащий в себе i-items, а T - количество транзакций. Т.е. в общем виде это показатель "частотности" данного itemset во всех анализируемых транзакциях. Но это касается только X. Нам же интересен скорее вариант, когда у нас в одном itemset встречаются x1 и x2 (например).
Ну тут тоже все просто. Пусть x1 = {Пиво}, а x2 = {Подгузники}, значит нам нужно посчитать, во скольких транзакициях втречаестя эта парочка. 

$supp(x_1\cup x_2) = \frac{\sigma(x_1 \cup x_2)}{|T|}$, где $\sigma $ - количество транзакций, содержащих $x_1$ и $x_2$

##### Confidence (уровень доверия)

Следующее ключевое понятие - confidence. Это показатель того, как часто наше правило срабатывает для всего датасета. 

$conf(x_1\cup x_2) = \frac{supp(x_1 \cup x_2)}{supp(x_1)}$

Приведем пример: мы хотим посчитать confidence для правила "кто покупает пиво, тот покупает и подгузники".

Для этого сначала посчитаем, какой support у правила "покупает пиво", потом посчитаем support у правила "покупает пиво и подгузники", и просто поделим одно на другое. Т.е. мы посчитаем в скольких случаях (транзакциях) срабатывает правило "купил пиво $supp(X)$, купил подгузники и пиво $supp(X \cup Y)$"
Ничего не напоминает? Байес смотрит на все это несколько недоуменно и с презрением:)

<img src="src/images/thomas-bayes.png">

##### Lift (подъем или повышение)

Следующее понятие в нашем списке - lift. Грубо говоря, lift - это отношение "зависимости" items к их "независимости". Lift показывает, насколько items зависят друг от друга. Это очевидно из формулы:

$lift(x_1\cup x_2) = \frac{supp(x_1 \cup x_2)}{supp(x_1) \times supp(x_2)}$

Например, мы хотим понять зависимость покупки пива и покупки подгузников. Для этого считаем support правила "купил пиво и подгузники" и делим его на произведение правил "купил пиво" и "купил подгузники". В случае, если lift = 1, мы говорим, что items независимы и правил совместной покупки тут нет. Если же lift > 1, то велечина, на которую lift, собственно, больше этой самой единицы, и покажет нам "силу" правила. Чем больше единицы, тем круче.

##### Conviction (убедительность)

В общем виде Conviction - это "частотность ошибок" нашего правила. Т.е., например, как часто покупали пиво без подгузников и наоборот.

$conv(x_1\cup x_2) = \frac{1 - supp(x_2)}{1 - conf(x_1 \cup x_2)}$

Чем результат по формуле выше ближе к 1, тем лучше. Например, если conviction покупки пива и подгузников вместе равен 1.2, это значит, что правило "купил пиво и подгузники" было бы в 1.2 раза (на 20%) более верным, чем если бы совпадение этих items в одной транзакции было бы чисто случайным. Немного неинтуитивное понятие, но оно и используестя н етак часто, как предыдущие три. 

Существует ряд часто используемых алгоритмов, позволяющих находить правила в itemsets согласно перечисленным выше понятиям - Наивный или брутфорс-алгоритм, Apriori- алгоритм, ECLAT-алгоритм, FP-growth алгоритм и другие.

### Брутфорс

Найти правила в itemsets в общем не сложно, сложно сделать это эффективно. Брутфорс-алгоритм самый простой и, в то же время, самый неэффективный способ.

В псевдо-коде алгоритм выглядит так:

**ВХОД**: Датасет $D$, содержащий список транзакций

**ВЫХОД**: Наборы itemsets $F_1, F_2, F_3, ... F_q$, где $F_i$ - набор itemsets размера $I$, которые встечаются как минимум $s$ раз в $D$

**ПОДХОД:**

1. $R \leftarrow$  целочистленный array, содержащий в себе все комбинации items в $D$, размера $2^{|D|}$
2. **for** $n \leftarrow  [1, |D|]$ **do**:
<br>
$F \leftarrow$  все возможные комбинации из $D_n$
<br>
Увеличить каждое значение в $R$ согласно значениям в каждом $F[]$
5. **return** Все itemsets в $R \geq s$

Сложность брутфорс-алгоритма очевидна:
Для того чтобы найти все возможные Association rules применяя брутфорс-алгоритм нам необходимо перечислить все подмножества $X$ из набора $I$ и для каждого подмножества $X$ рассчитать $supp(X)$. Данный подход будет состоять из следующих шагов:
-  генерация **кандидатов**. Данный шаг состоит из перебора всех возможных подмножеств $X$ подмножества $I$. Данные подмножества называются **кандидатами**, поскольку каждое подмножество теоретически может являться часто встречаемым подмножеством, то множество потенциальных кандидатов будет состоять из $2^{|D|}$ элементов (здесь $|D|$ обозначается число элементов множества $I$, а $2^{|D|}$ часто называется булеаном множества $I$, то есть множество всех подмножеств $I$)
-  расчет **support**. На данном шаге рассчитывается $supp(X)$ каждого кандидата $X$

Таким образом, сложность нашего алгоритма будет: $O(|I|*|D|*2^{|I|})$, где
<br>$O(2^{|I|})$ - количество возможных кандидатов</br>
<br>$O(|I|*|D|)$ - сложность расчета sup(X). Поскольку для расчета  $supp(X)$ нам необходимо перебрать все элементы в $I$ в каждой транзакции $t \in T$</br>

В нотации O-большое нам понадобится $O(N)$ операций, где $N$ - количество itemsets.
Таким образом, для применения брутфорс-алгоритма нам понадобится $2^i$ ячеек памяти, где $i$ - индивидуальный itemset. Т.е. для хранения и обсчета 34 items нам понадобится 128GB RAM (для 64-битных целосчисленных ячеек). 
<br>Таким образом брутфорс поможет нам в обсчете транзакций в палатке с шаурмой у вокзала, но для чего-то более серьезного он совершенно не пригоден. Реализацию в Питоне и R деалть нет смысла.

### Apriori Algorithm

#### Теория

Используемые понятия:
<br> -  Множество объектов (**itemset**): $X \subseteq I = \{x_1, x_2, ..., x_n\}$</br>
<br> -  Множество идентификаторов транзакций (**tidset**): $T = \{t_1, t_2, ..., t_m\}$</br>
<br> -  Множество транзакций (**transactions**): $\{(t,\ X):\ t\in T,\ X \in I\}$</br>

Введем дополнительно еще несколько понятий.
<br>Будем рассматривать дерево префиксов (prefix tree), где 2 элемента $X$ и $Y$ соединены, если $X$ является прямым подмножеством $Y$. Таким образом мы можем пронумеровать все подмножества множества $I$. Рисунок приведен ниже</br>

<img src="/src/images/tree_1.png">

Итак, рассмотрим Apriori алгоритм.
<br>
Apriori алгоритм использует следующее утверждение: если $X \subseteq Y$, то $supp(X) \geq supp(Y)$. Откуда следуют следующие 2 свойства:
 -  если $Y$ встречается часто, то любое подмножество $X: X \subseteq Y$  так же встречается часто
 -  если $X$ встречается редко, то любое супермножество $Y: Y \supseteq X$ так же встречается редко
 </br>

Apriori алгоритм по-уровнево проходит по префиксному дереву и рассчитывает частоту встрчаемости подмножеств $X$ в $D$. Таким образом, в соответствии с алгоритмом:
 -  исключаются редкие подмножества и все их супемножества
 -  рассчитывается $supp(X)$ для кадого подходящего кандидата $X$ размера $k$ на уровне $k$

<img src="/src/images/tree_2.png">

В псевдо-код нотации Априори-алгоритм выглядит следующим образом:

**ВХОД**: Датасет $D$, содержащий список транзакций, и $\sigma$ - задаваемый пользователем порог $supp$

**ВЫХОД**: Список itemsets $F(D, \sigma)$

**ПОДХОД**:

1. $C_1$ $\leftarrow$ $[{i}|i \in J]$
2. $k \leftarrow 1$
3. **while** $C_k \neq 1$ **do**:
4. #Считаем все support для всех кандидатов
<br>
**for all** транзакций (tid, I) $\in D$ **do**:
<br>
    **for all** кандидатов $X \in C_k$ **do**:
<br>
**if** $X \subseteq I$:
<br>
$X.support++$
5. #Вытаскиваем все частые itemsets:
<br>
$F_k = $ {X|X.support > $\sigma$}
6. #Генерируем новых кандидатов
<br>
**for all** $X,Y \in F_i, X[i] = Y[i]$ **for** $1 \leq i \leq k-1$ **and** $X[k] \leq Y[k]$ **do**:
<br>
$I = X \cup$ {$Y$|$k$|}
<br>
**if** $\forall J \subset I,|J| = k: J \in F_k$ **then**
<br>
$C_k+1$ $\leftarrow C_k+1 \cup I$
<br>
$k++$


Таким образом, Apriori-адгоритм сначала ищет все единичные (содержащие 1 элемент) itemsets, удовлетворяющие заданному пользователем $supp$, затем составляет из них пары такие по принципу иерархической монотонности, т.е. если $x_1$ встречается часто и $x_2$ встречается часто, то и $[x_1, x_2]$ встречается часто.

Явным минусом такого подхода является то, что необходимо "просканировать" весь датасет, посчитать все $supp$ на всех уровнях breadth-first search (поиск в ширину - подробнее [тут](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%B8%D1%81%D0%BA_%D0%B2_%D1%88%D0%B8%D1%80%D0%B8%D0%BD%D1%83))
Это также может подъесть RAM на больших датасетах, зотя и намного эффективнее брутфорса

#### Реализация в Python

from sklearn import... эммм... а импортировать-то и нечего:) На данный момент модулей для ALR в sklearn нет. Ну ничего, погуглим или напишем свои, правда?

По сети гуляет целый ряд реализаций, например [вот](https://github.com/asaini/Apriori), [вот](http://adataanalyst.com/machine-learning/apriori-algorithm-python-3-0/), и даже [вот](https://codereview.stackexchange.com/questions/38101/apriori-algorithm-using-pandas)

Мы же на практике придерживаемся алгоритма apyori, написанного Ю Мочиузки (Yu Mochizuki). Полный код приводить не будем, желающие могут посмотреть [тут](https://github.com/ymoch/apyori) , а вот архитектуру решения и пример использования покажем.

Условно решение Мочизуки можно разделить на 4 части: Структура данных, Внутренние функции, API и Прикладные функции.

Первая часть модуля (Структура данных) работает с изначальным датасетом. Реализуется класс TransactionManager, методы которого объединяют транзакции в матрицу, формируют список правил-кандидатов и считают support для каждого правила. Внутренние функции дополнительно по support'у формируют списки правил и соответственно их ранжируют. API логично позволяет работать напрямую с датасетами, а Прикладные функции позволяют обрабатывать транзакции и выводить результат в читаемый вид. Никакого rocketscience.  

Посмотрим, как использовать модуль на реальном (ну, в данном случае - игрушечном) датасете.

In [13]:
# подгрузим модули
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyfim

In [6]:
# загрузим данные
dataset = pd.read_csv('src/data/Market_Basket_Optimisation.csv', header = None)

In [15]:
transactions = dataset.values.tolist()

In [ ]:
transactions

In [37]:
#%%time
#lst = []
pyfim.fim(transactions, target='s', supp=2, zmin=1, zmax=3)

[(('almonds', nan), 152),
 (('almonds',), 153),
 (('meatballs', nan), 157),
 (('meatballs',), 157),
 (('strawberries', nan), 160),
 (('strawberries',), 160),
 (('fresh tuna', nan), 167),
 (('fresh tuna',), 167),
 (('french wine', nan), 169),
 (('french wine',), 169),
 (('oil', nan), 173),
 (('oil',), 173),
 (('muffins', nan), 181),
 (('muffins',), 181),
 (('cereals', nan), 193),
 (('cereals',), 193),
 (('vegetables mix', nan), 192),
 (('vegetables mix',), 193),
 (('ham', nan), 199),
 (('ham',), 199),
 (('pepper', nan), 199),
 (('pepper',), 199),
 (('energy drink', nan), 199),
 (('energy drink',), 200),
 (('energy bar', nan), 203),
 (('energy bar',), 203),
 (('light mayo', nan), 204),
 (('light mayo',), 204),
 (('yogurt cake', nan), 205),
 (('yogurt cake',), 205),
 (('red wine', nan), 211),
 (('red wine',), 211),
 (('whole wheat pasta', nan), 221),
 (('whole wheat pasta',), 221),
 (('butter', nan), 226),
 (('butter',), 226),
 (('tomato juice', nan), 227),
 (('tomato juice',), 228),
 (('

In [7]:
#dataset.describe #всего 7500 записей-транзакций

In [8]:
# посомтрим на датасет
dataset.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
1,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Видим, что датасет у нас представляет разреженную матрицу, где в строках у нас набор items в каждой транзакции.

In [9]:
#создаим из них матрицу
transactions = []
for i in range(0, 7501): 
    transactions.append([str(dataset.values[i,j]) for j in range(0, 20)])

In [10]:
#загружаем apriori
import apyori

In [11]:
%%time
# и обучимся правилам. Обратите внимание, что пороговые значения мы вибираем сами в зависимости от того, /
# насколкьо "сильные" правила мы хотим получит
# min_support -- минимальный support для правил (dtype = float).
# min_confidence -- минимальное значение confidence для правил (dtype = float)
# min_lift -- минимальный lift (dtype = float)
# max_length -- максимальная длина itemset (вспоминаем про k-itemset)  (dtype = integer)

result = list(apyori.apriori(transactions, min_support = 0.003, min_confidence = 0.2, min_lift = 3, min_length = 2))

Wall time: 5.72 s


In [ ]:
# вот с визуализацией в пакете проблемы, точнее геморрой....

In [ ]:
import shutil, os 

In [ ]:
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO

In [ ]:
import json #преобразовывать будем в json, используя встроенные в модуль методы

In [ ]:
output = []
for RelationRecord in result:
    o = StringIO()
    apyori.dump_as_json(RelationRecord, o)
    output.append(json.loads(o.getvalue()))
data_df = pd.DataFrame(output)

In [ ]:
# и взгялнем на итоги
pd.set_option('display.max_colwidth', -1)

from IPython.display import display, HTML

display(HTML(data_df.to_html()))

Итого мы видим:

1. Пары items
2. items_base - первый элемент пары
3. items_add - второй (добавленный алгоритмом) элемент пары
4. confidence - значение confidence для пары
5. lift - значение lift для пары
6. support - начение support для пары. При желании, по нему можно отсортировать 


Результаты логичные: эскалоп и макароны, эскалоп и сливочно-грибной соус, курица и нежирная сметана, мягкий сыр и мед и т.д. - все это вполне логичные и, главное, вкусные сочетания:)

#### Реализация в R

ARL тот случай,скогда R-фаги могут злорадно похихикать. В R реализована библиотека arules, где присутствует и apriori, и другие алгоритмы. Официальную доку можно посмотреть [тут](https://cran.r-project.org/web/packages/arules/arules.pdf)

Посмотрим на нее в действии:

Для начала установим ее (если еще не установили):
<br>
install.packages('arules')

Считаем данные и преобразуем их в матрицу транзакций:
<BR>
<br>
$library(arules)$
<br>
$dataset = read.csv('Market_Basket_Optimisation.csv', header = FALSE)$
<br>
$dataset = read.transactions('Market_Basket_Optimisation.csv', sep = ',', rm.duplicates = TRUE)$

Посмотрим на данные:
<BR>
$summary(dataset)$
<br>
$itemFrequencyPlot(dataset, topN = 10)$

Выучим наши правила:
<br>
В общем виде фнкция вызова apriori выглядит так:  $apriori(data, parameter = NULL, appearance = NULL, control = NULL)$, где
<br>

$data$ - наш датасет
<br>
$paramter$ - список (list) параметров для модели: минимальные support, confidence и lift
<br>
$appearance$ - отвечает за отображение данных. Может принимать значения lhs, rhs, both, items, none, которые определяют положение items в output
<br>
$control$ - отвечает за сортировку вывода (ascending, descending, без сортировки), а также за то, отображать ли прогрессбар или нет (параметр verbose)



Обучим модель:

$rules = apriori(data = dataset, parameter = list(support = 0.004, confidence = 0.2))$

И посмотрим на результаты:
<br>
$inspect(sort(rules, by = 'lift')[1:10])$

Убедимся, что на выходе имеем примерно те же результаты, что при использовании модуля apyori в Python:
<br>

1. {light cream}                               => {chicken}       0.004532729
2. {pasta}                                     => {escalope}      0.005865885
3. {pasta}                                     => {shrimp}        0.005065991
4. {eggs,ground beef}                          => {herb & pepper} 0.004132782
5. {whole wheat pasta}                         => {olive oil}     0.007998933
6. {herb & pepper,spaghetti}                   => {ground beef}   0.006399147
7. {herb & pepper,mineral water}               => {ground beef}   0.006665778
8. {tomato sauce}                              => {ground beef}   0.005332622
9. {mushroom cream sauce}                      => {escalope}      0.005732569
10. {frozen vegetables,mineral water,spaghetti} => {ground beef}   0.004399413

Как видно, в R apriori использовать на данный момент намного удобнее, чем в Python.

### ECLAT Algorithm

#### Теория

Идея алгоритма ECLAT (Equivalence CLAss Transformation) заключается в ускорении подсчета $supp(X)$. Для этого нам необходимо проиндексировать наше базу данных $D$ так, чтобы это позволило быстро рассчитывать $supp(X)$

Легко заметить, что если $t(X)$ обозначает множество всех транзакций, где встречается подмножество $X$, то
<br>$t(XY) = t(X) \cap t(Y)$</br>
<br>и</br>
<br>$supp(XY) = |t(XY)|$</br>
<br> то есть $supp(XY)$ равен кардинальности (размеру) множества $t(XY)$

<img src="/src/images/tree_3.png">

Данный подход может быть значительно усовершенствован путем уменьшения размера промежуточных множеств идентификаторов транзакций (tidsets). А именно, мы можем хранить не все множество транзакций на промежуточном уровне, а только множество различий этих транзакций. Предположим, что
<br>$X_a = \{x_1, x_2,..., x_{n-1}, a\}$</br>
<br>$X_b = \{x_1, x_2,..., x_{n-1}, b\}$</br>
<br>Тогда, мы получим: </br>
<br>$X_{ab} = \{x_1, x_2,..., x_{n-1}, a, b\}$</br>
<br>$diffset(X_{ab})$ это множество всех id транзакций, которыесодержат префикс $X_a$ но не содержат элемент $b$: </br>
<br>$d(X_{ab}) =t(X_a)/t(X_{ab})=t(X_a)/t(X_{b})$</br>

<img src="/src/images/tree_4.png">

В отличие от Apriori-алгоритма, ECLAT производите поиск в глубину (DFS, [подробнее тут](https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%B8%D1%81%D0%BA_%D0%B2_%D0%B3%D0%BB%D1%83%D0%B1%D0%B8%D0%BD%D1%83)). Иногда его называют "вертикальным" (в отличие от "горизонтального" для Apriori)

**ВХОД**: Датасет $D$, содержащий список транзакций, $\sigma$ - задаваемый пользователем порог $supp$ и новый элемент префикс $I \subseteq J$

**ВЫХОД**: Список itemsets $F[I](D, \sigma)$ для соответсвующего префикса $I$

**ПОДХОД:**

1. $F[i] \leftarrow$ {}
<br>
2. **for** all $i \in J$ in $D$ **do**:
<br>
$F[I] := F[I] \cup$ {I $\cup$ {i}} 
<br>
3. #Создаем $D_i$
<br>
$D_i \leftarrow$ {}
<br>
**for** all $j \in J$ in $D$ таких, что $j > i$ **do:**
<br>
$C \leftarrow$ покрывает ({$i$} $\cap$ покрывает {$j$})
<br>
**if** $|C| \geq \sigma$ **then**
<br>
$D_i \leftarrow D_i \cup$ {$C$,$i$}
<br>
4. #DFS - рекурсия:
<br>
Считаем $F|I \cup$ {$i$}| $(D_i, \sigma)$
<br>
$F[I]: F[I] \cup F[I \cup i]$


Ключевым понятием для ECLAT-алгоритма является I-префикс. В началае генерируется пустое множество I, это позволяет нам на первом проходе выделить вче частотные itemsets. Затем алгоритм будет вызывать сам себя и увеличивать I на 1 на каждом шаге до тех пор, пока не будет достигнута заданная пользователем длина I. 

Для хранения значений используется прфеиксное дерево (trie (не tree:)), [тут подробнее](https://ru.wikipedia.org/wiki/%D0%9F%D1%80%D0%B5%D1%84%D0%B8%D0%BA%D1%81%D0%BD%D0%BE%D0%B5_%D0%B4%D0%B5%D1%80%D0%B5%D0%B2%D0%BE)). Вначале строится нулевой корень дерева (то самое пустое множество I), затем по мере прохода по itemsets алгоритм прописывает содержащиеся в каждом itesmsets items, при этом самая левая ветвь является child нулевого корня и далее вниз. При этом ветвей столько, сколкьо items встречаестя в itemsets. Такой подход позволяет записывать itemset в памяти толкьо один раз, что делает ECALT быстрее Apriori. 

<<**КАРТИНКА**>>

#### Реализация в Python

In [34]:
def createC1(dataset):
    "Create a list of candidate item sets of size one."
    c1 = []
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()
    #frozenset because it will be a ket of a dictionary.
    return map(frozenset, c1)

In [35]:
def scanD(dataset, candidates, min_support):
    "Returns all candidates that meets a minimum support level"
    sscnt = {}
    for tid in dataset:
        for can in candidates:
            if can.issubset(tid):
                sscnt.setdefault(can, 0)
                sscnt[can] += 1

#### Реализация в R

И вновь пользователи R ликуют, для них никаких танцев с бубном делать не надо, все по аналогии с apriori. 

Запускаем библиотеку и читаем данные:

$library(arules)$
<br>
$dataset = read.csv('Market_Basket_Optimisation.csv')$
<br>
$dataset = read.transactions('Market_Basket_Optimisation.csv', sep = ',', rm.duplicates = TRUE)$


Быстрый взгляд на датасет:
<br>
$summary(dataset)$
<br>
$itemFrequencyPlot(dataset, topN = 10)$

Правила:
<br>
$rules = eclat(data = dataset, parameter = list(support = 0.003, minlen = 2))$ 
<br>
Обратите внимание, настраиваем толкьо support и минимальную длину (k в k-itemset)

И смотрим на результаты:
<br>
$inspect(sort(rules, by = 'support')[1:10])$

### FP-Growth Algorithm

#### Теория

FP-Growth (Frequent Pattern Growth) алгоритм самый молодой из нашей троицы, впервые он описан в 2000 году в [7].
FP-Growth предлагает радикальную вещь - отказатсья от генерации **кандидатов** (напомним, шенерация кандидатов лежит в основе Apriori и ECLAT). Теоретчиески, такой подход позволит еще больше увеличитьь скорость алгоритма и использовать еще меньше памяти.

Это достигаестя за счет хранения в памяти префиксного дерева (trie) не из комбинаций кандидатов, а из самих транзакций. 
При этом FT-Growth генерирует таблицу заголовков дял каждого item, чей $supp$ выше заданного пользователем. Эта таблица заголвоков хранит связанный спсико всех однотипных узлов префиксного дерева. Таким образом, алгоритм сочетает с ебе плюсы **BFS** за счет таблицы заголовков и **DFS** за счет построения trie. Псевдокод алгоритма схож с ECALT, за некоторыми исключениями.

**ВХОД**: Датасет $D$, содержащий список транзакций, $sigma$ - задаваемый пользователем порог $supp$ и префикс $I \subseteq J$

**ВЫХОД**: Список itemsets $F[I](D, sigma)$ для соответсвующего префикса $I$

**ПОДХОД:**

1. $F[i] \leftarrow$ {}
<br>
2. **for** all $i \in J$ in $D$ **do**:
<br>
$F[I] := F[I] \cup$ {I $\cup$ {i}} 
<br>
3. #Создаем $D_i$
<br>
$D_i \leftarrow$ {}
<br> 
**$H_i \leftarrow$ {}**
<br>
**for** all $j \in J$ in $D$ таких, что $j > i$ **do:**
<br>
**if** $supp$ ($I \cup$ {$i$,$j$}) $\geq \sigma$ **then**:
<br>
$H \leftarrow H \cup$ {$j$}
<br>
**for** all $(tid, X) \in D$ при $I \in X$ **do**:
<br>
$D_i \leftarrow D_i \cup$ ({$tid,X \cap H$})
<br>
4. #DFS - рекурсия:
<br>
Считаем $F|I \cup$ {$i$}| $(D_i, \sigma)$
<br>
$F[I] \leftarrow F[I] \cup F[I \cup i]$


#### Реализация в Python

Реализации FP-Growth в Питоне повезло не больше, чем другим ALR-алгоритмам. Стандартных библиотек под него нет.

Неплохо FP_Growth представлен в pyspark, смотреть тут - http://spark.apache.org/docs/2.2.0/mllib-frequent-pattern-mining.html

На gitHub тоже можно найти несколкьо решений эпохи неолита, например тут - https://github.com/enaeseth/python-fp-growth и тут - https://github.com/evandempsey/fp-growth

Потестим второй вариант

In [1]:
!pip install pyfpgrowth

  Running setup.py bdist_wheel for pyfpgrowth: started
  Running setup.py bdist_wheel for pyfpgrowth: finished with status 'done'
  Stored in directory: C:\Users\pavel.golubev\AppData\Local\pip\Cache\wheels\b0\77\ee\b8139db041c69f769d35de3373fe03129c25ef186521e924f1
Successfully built pyfpgrowth


In [2]:
import pyfpgrowth

In [23]:
%%time
patterns = pyfpgrowth.find_frequent_patterns(transactions, 3)

Wall time: 29.6 s


In [24]:
%%time
rules = pyfpgrowth.generate_association_rules(patterns, 0.6)

Wall time: 6.41 s


In [26]:
rules;

#### Реализация в R

В данном случае R не отстает от Питона: в такой удобнйо и родной arules FP-Growth отсутствует. 

В то же время, как и для Питона, реализация сущетсвует в Spark - https://spark.apache.org/docs/2.2.0/api/R/spark.fpGrowth.html

## Заключение

В заключении давайте эмпирически сравним **эффектвиность** метрикой *runtime* в зависимости от плотности датасета и длин транзакций датасета. 

Под *плотностью* датасета подразумеваестя отношение транзакций, содержащих в себе частые items к общему количеству транзакций.

<img src="src/images/dens_run.png">

Из графика очевидно, что эффективность (чем меньше runtime, тем эффективнее) Apriori-алгоритма падает при увеличении плотности датасета.

Под *длинной транзакции* понимается количество в items в itemset

<img src="src/images/size_run.png">

Очевидно, что при увеличении длины транзакции Apriori также справляется гораздо хуже.

In [20]:
import random

def sizeof_fmt(num):
    for x in ['','k','m','g']:
        if num < 1000.0:
            return "%3.1f%s" % (num, x)
        num /= 1000.0
    return "%3.1f%s" % (num, 't')

# These paramaters can be changed to determine the type of data to generate.
# How many baskets, or transactions
BASKET_COUNT = 10000000
# Maximum number of items per basket
MAX_ITEMS_PER_BASKET = 7
# The number of unique items
ITEM_COUNT = 200
# The number of frequent itemsets
NUM_FREQUENT_ITEMSETS = 2
# The probability of a basket containing a frequent itemset.
PROB_FREQUENT = 0.5

# Generate the data
pop_frequent = ["F"+str(n) for n in range(0,MAX_ITEMS_PER_BASKET)]
pop_regular = ["I"+str(n) for n in range(MAX_ITEMS_PER_BASKET,ITEM_COUNT)]


freq_itemsets = []

for i in range(NUM_FREQUENT_ITEMSETS):
    cnt = random.randint(1,MAX_ITEMS_PER_BASKET)
    freq_itemsets.append(random.sample(pop_frequent,cnt))


In [21]:
# Create a filename that encodes the MAX_ITEMS_PER_BASKET and BASKET_COUNT into
# the filename.
with open('C:/Users/pavel.golubev/Desktop/arules_shared/src/data/fprob'+str(PROB_FREQUENT)+"_tsz"
        +str(MAX_ITEMS_PER_BASKET)+'_tct'
        +sizeof_fmt(BASKET_COUNT)+'.txt', 'w') as f:
    for i in range(BASKET_COUNT):
        line = []

        cnt = random.randint(1,MAX_ITEMS_PER_BASKET)
        if random.random()<=PROB_FREQUENT:
            idx = random.randint(0,len(freq_itemsets)-1)
            for j in range(len(freq_itemsets[idx])):
                line.append(freq_itemsets[idx][j])

        needed = max(0,cnt - len(line))
        line = line + random.sample(pop_regular,needed)

        f.write(" ".join(line)+"\n")

### Итоги

Итак, мы познакомились с базовой теорией ARL ("кто купил х, также купил y") и основными понятиями и метриками (support, confidence, lift и conviction).

Посмотрели 3 самых популярных алгоритма (Apriori, ECLAT, FP-Growth), позавидовали пользователям R и библиотеки arules, попробовали сами реализовать ECALT.

Основные моменты:
1. ARL лежат в основе рекомендательных систем
2. ARL широко применимы - от традиционного ритейла и онлайн ритейла (от Ozon до Steam) до банков и телекома (подключаемые сервисы и услуги)
3. ARL относительно легко использовать, существуют реализации разного уровня проработки для разных задач.
4. ARL хорошо интепретируются и не требуют специальных навыков

Помимо рассмотренных бызовых алгоритмов существет модификации и ответвления:

1. AprioriDP (Deep Programming) - позволяет хранить $supp$ в специальной структруе данных, работает немного быстрее классичсекого Apriori
2. FP Bonsai - улучшенный FP-Growth с обрезкой префиксного дерева (пример алгоритма с **ограничениями**
3. Серия Multi-Relations алгоритмов, которые определают несколкьо свзяей в itemsets.
4. и другие

В заключении не можем не упомянуть о сумрачном гении ARL докторе Кристиане Боргельте из Университета Констанца (http://www.borgelt.net/software.html).
На самом деле все, с чем мы мучались на протяжении этой статьи им давно уже реализовано на С, Python, Java и R. Ну или почти все. Существет даже GUI за его авторством, где в пару кликов можно загрузить датасет, выбрать нужный алгоритм и найти правила. Это при условии, что оно у вас заработает:)
Для простых же задач достаточно и того, что мы рассмотрели в этой статье. А если недостаточно - призываем писать реализацию самим! 

**Использованная литература:**
<br>
[1] Discovery, analysis and presentation of strong rules. G. Piatetsky-Shapiro. Knowledge Discovery in Databases, AAAI Press, (1991)
<br>
[2] Mining Association Rules between Sets of Items in Large Databases http://arbor.ee.ntu.edu.tw/~chyun/dmpaper/agrama93.pdf
<br>
[3] Fast Algorithms for Mining Association Rules http://www.vldb.org/conf/1994/P487.PDF
<br>
[4] Ask Dan! http://www.dssresources.com/newsletters/66.php
<br>
[5] Introduction to arules – A computational environment for mining association rules and frequent item sets http://www.lsi.upc.edu/~belanche/Docencia/mineria/Practiques/R/arules.pdf
<br>
[6] Публикации Д-ра Боргельта - http://www.borgelt.net/publications.html
<br>
[7] J. Han, J. Pei, and Y. Yin, “Mining frequent patterns without candidate generation,” in ACM SIGMOD Record, vol. 29, no. 2. ACM, 2000, pp. 1–12.
<br>
[8] Shimon, Sh. Improving Data mining algorithms using constraints. The Open University of Israel, 2012.